### <font color = 'Red'> Project 1: Data Wrangling and Regression Analysis
**PART 1:** Data Collection and Preparation
    1. Read the process the Fortune1000 data file
    2. Add a column of Company URLs to the DataFrame
    3. Extract/Scrape Search Advertising Related Data from A website

In [143]:
# import packages

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as beaut
import requests
import re
import warnings
from colorama import Fore, Style

In [20]:
# read txt file

with open("fortune1000.txt","r") as file:            # file to refer to object
    dataContent = file.readlines()                   # read the data into dataContent line by line
fileList = [line.strip() for line in dataContent]    # make a list with individual elements from newline
del fileList[-1]                                     # delete extra single extra last element
dataArr = np.array(fileList)                         # make array

In [51]:
# make dataframe
fortune_df = pd.DataFrame(dataArr.reshape(1000,11), columns = ['Rank','Name', 'Revenue($m)', '% Change in revenue', 
                                                        'Profits($m)', '% Change in Profits', 'Assets($m)',
                                                        'Market Value($m)', 'Change in Rank (1000)', 'Employees',
                                                        'Change in Rank (500 Only)'])

In [52]:
# drop extra columns
fortune_df = fortune_df.drop(['% Change in revenue', '% Change in Profits',
                              'Change in Rank (1000)','Change in Rank (500 Only)'], axis=1)

In [53]:
# df specifications

#fortune_df.shape               # column row count
fortune_df.head()              # data peak
#fortune_df.describe()          # statistic details
#fortune_df.info()              # column list
#fortune_df.isna().any()        # check NA

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees
0,1,Walmart,"$523,964","$14,881","$236,495","$321,803.3","2,200,000"
1,2,Amazon.com,"$280,522","$11,588","$225,248","$970,680.1","798,000"
2,3,Exxon Mobil,"$264,938","$14,340","$362,597","$160,696.3","74,900"
3,4,Apple,"$260,174","$55,256","$338,516","$1,112,640.8","137,000"
4,5,CVS Health,"$256,776","$6,634","$222,449","$77,375.8","290,000"


In [54]:
# clean data to remove '$' and ','
fortune_df = fortune_df.replace('\$|,', '', regex=True)
fortune_df = fortune_df.replace('-', np.nan, regex=True)

### <font style = 'italic'> eof

In [55]:
# export processed sample set
fortune_df.to_csv('fortune_df.csv')

In [56]:
# read processed sample set
fortune_df = pd.read_csv("fortune_df.csv")
fortune_df = fortune_df.drop(fortune_df.columns[[0]],axis=1)

In [57]:
# df specifications

#fortune_df.shape               # column row count
#fortune_df.head()              # data peak
#fortune_df.describe()          # statistic details
fortune_df.info()               # column list
#fortune_df.isna().any()        # check NA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              1000 non-null   int64  
 1   Name              969 non-null    object 
 2   Revenue($m)       1000 non-null   float64
 3   Profits($m)       854 non-null    float64
 4   Assets($m)        1000 non-null   float64
 5   Market Value($m)  950 non-null    float64
 6   Employees         1000 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 54.8+ KB


**Part 2:**

In [127]:
# read csv file
fortuneURLs_df = pd.read_csv("fortuneURL_df.csv")

# drop extra columns
fortuneURLs_df = fortuneURLs_df.drop(['Unnamed: 0'], axis=1)
# fortuneURLs_df = fortuneURLs_df.drop(['Rank', 'Name','Revenue($m)','Profits($m)','Assets($m)','Market Value($m)','Employees'], axis=1)

# drop rows with null URLs
# fortuneURLs_df = fortuneURLs_df.dropna(axis=0, subset=['URLs'])

# reset index
fortuneURLs_df = fortuneURLs_df.reset_index(drop=True)

In [128]:
# df specifications

#fortuneURLs_df.shape               # column row count
#fortuneURLs_df.head()              # data peak
#fortuneURLs_df.describe()          # statistic details
fortuneURLs_df.info()              # column list
#fortuneURLs_df.isna().any()        # check NA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              423 non-null    int64  
 1   Name              423 non-null    object 
 2   Revenue($m)       423 non-null    float64
 3   Profits($m)       390 non-null    float64
 4   Assets($m)        423 non-null    float64
 5   Market Value($m)  395 non-null    float64
 6   Employees         423 non-null    int64  
 7   URLs              423 non-null    object 
dtypes: float64(4), int64(2), object(2)
memory usage: 26.6+ KB


In [129]:
# add webiste outlook site url for scraping

fortuneURLs_df['wout'] = fortuneURLs_df['URLs'] + '.websiteoutlook.com'

In [130]:
# add empty columns to collect website attributes

fortuneURLs_df['alexaRank'] = ""
fortuneURLs_df['backlinks'] = ""
fortuneURLs_df['pageAuthority'] = ""
fortuneURLs_df['domainAuthority'] = ""
fortuneURLs_df['mozRank'] = ""
fortuneURLs_df['pageviews'] = ""
fortuneURLs_df['worth'] = ""

fortuneURLs_df['pageSize'] = ""
fortuneURLs_df['codeToTextRatio'] = ""

fortuneURLs_df['semrushRank'] = ""
fortuneURLs_df['keywords'] = ""
fortuneURLs_df['organicTraffic'] = ""
fortuneURLs_df['cost'] = ""
fortuneURLs_df['adwordsKeywords'] = ""
fortuneURLs_df['adwordsTraffic'] = ""
fortuneURLs_df['adwordsBudget'] = ""

fortuneURLs_df['is_run'] = 0

**Single website Run:** Print Check for all attributes

In [118]:
# website outlook URL for test run
print(fortuneURLs_df['wout'][0])

http://www.walmart.com.websiteoutlook.com


In [119]:
# beaut object creation
response = requests.get(fortuneURLs_df['wout'][1])
soup = beaut(response.content, "html.parser")

In [120]:
print('alexaRank: ' + soup.find('span',{"class":"label label-primary"}).get_text().strip())
print('backlinks: ' + soup.find('span',{"class":"label label-default"}).get_text().strip())
print('pageAuthority: ' + soup.find('span',{"class":"label label-info"}).get_text().strip())
print('domainAuthority: ' + soup.find('span',{"class":"label label-info"}).get_text().strip())
print('mozRank: ' + soup.find('span',{"class":"label label-info"}).get_text().strip())
print('pageviews: ' + soup.find('span',{"class":"label label-warning"}).get_text().strip())
print('worth: ' + soup.find('span',{"class":"label label-danger"}).get_text().strip())

print('pageSize: ' + soup.find("dt",text="Page Size").findNext("dd").string.strip())
print('codeToTextRatio: ' + soup.find("dt",text="Code to Text Ratio").findNext("dd").text.split('(')[0].strip())

print('semrushRank:' + soup.find("td",text="Semrush Rank").findNext("td").text.strip())
print('keywords:' + soup.find("td",text="Keywords").findNext("td").text.strip())
print('organicTraffic:' + soup.find("td",text="Organic Traffic").findNext("td").text.strip())
print('cost:' + soup.find("td",text="Cost (in USD)").findNext("td").text.strip())
print('adwordsKeywords:' + soup.find("td",text="Adwords Keyword").findNext("td").text.strip())
print('adwordsTraffic:' + soup.find("td",text="Adwords Traffic").findNext("td").text.strip())
print('adwordsBudget:' + soup.find("td",text="Adwords budget (in USD)").findNext("td").text.strip())

alexaRank: 14
backlinks: 0
pageAuthority: 0/100
domainAuthority: 0/100
mozRank: 0/100
pageviews: 78.57M/ Day
worth: 172.07M
pageSize: 64Kb
codeToTextRatio: 0.27%
semrushRank:3
keywords:91384723
organicTraffic:950416767
cost:724598977$
adwordsKeywords:79995
adwordsTraffic:2943519
adwordsBudget:9398034$


**All URL Runs:**

_With time delay inserted between each run to avoid timeout exceptions_

In [151]:
warnings.filterwarnings('ignore')
runner = 0

for i in range(len(fortuneURLs_df['wout'])):

    # check URL exists
    if not fortuneURLs_df['wout'][i]:
        continue
        
    # print url for tracking
    print(str(i) + ' ' + fortuneURLs_df['wout'][i], end = '')
    
    # check port errors
    if fortuneURLs_df['is_run'][i] == -1:
        print(f'{Fore.RED}\033[1m: Not Run: Port Errors{Style.RESET_ALL}')
        continue
    
    # check is run flag
    if fortuneURLs_df['is_run'][i] >= 1:
        print(f'{Fore.GREEN}\033[1m: Not Run: Exists{Style.RESET_ALL}')
        continue
                
    # beaut object creation
    try:
        response = requests.get(fortuneURLs_df['wout'][i])
        soup = beaut(response.content, "html.parser")
        response.close() 
        runner += 1
    except:
        fortuneURLs_df['is_run'][i] = -1
        print(f'{Fore.RED}\033[1m: New Port Exception{Style.RESET_ALL}')
        continue

    # append data to columns if not null
    try:
        fortuneURLs_df['alexaRank'][i] = soup.find('span',{"class":"label label-primary"}).get_text().strip()
        fortuneURLs_df['backlinks'][i] = soup.find('span',{"class":"label label-default"}).get_text().strip()
        fortuneURLs_df['pageAuthority'][i] = soup.find('span',{"class":"label label-info"}).get_text().strip()
        fortuneURLs_df['domainAuthority'][i] = soup.find('span',{"class":"label label-info"}).get_text().strip()
        fortuneURLs_df['mozRank'][i] = soup.find('span',{"class":"label label-info"}).get_text().strip()
        fortuneURLs_df['pageviews'][i] = soup.find('span',{"class":"label label-warning"}).get_text().strip()
        fortuneURLs_df['worth'][i] = soup.find('span',{"class":"label label-danger"}).get_text().strip()
        fortuneURLs_df['is_run'][i] = 1
        print(f'{Fore.GREEN}\033[1m: 1...{Style.RESET_ALL}', end = '')
    except:
        print(f'{Fore.RED}\033[1m: New Scraping Exception 1st Block{Style.RESET_ALL}')
        continue

    try:
        fortuneURLs_df['pageSize'][i] = soup.find("dt",text="Page Size").findNext("dd").string.strip()
        fortuneURLs_df['codeToTextRatio'][i] = soup.find("dt",text="Code to Text Ratio").findNext("dd").text.split('(')[0].strip()
        fortuneURLs_df['is_run'][i] = 2
        print(f'{Fore.GREEN}\033[1m: 2...{Style.RESET_ALL}', end = '')
    except:
        print(f'{Fore.RED}\033[1m: New Scraping Exception 2nd Block{Style.RESET_ALL}')
        continue

    try:
        fortuneURLs_df['semrushRank'][i] = soup.find("td",text="Semrush Rank").findNext("td").text.strip()
        fortuneURLs_df['keywords'][i] = soup.find("td",text="Keywords").findNext("td").text.strip()
        fortuneURLs_df['organicTraffic'][i] = soup.find("td",text="Organic Traffic").findNext("td").text.strip()
        fortuneURLs_df['cost'][i] = soup.find("td",text="Cost (in USD)").findNext("td").text.strip()
        fortuneURLs_df['adwordsKeywords'][i] = soup.find("td",text="Adwords Keyword").findNext("td").text.strip()
        fortuneURLs_df['adwordsTraffic'][i] = soup.find("td",text="Adwords Traffic").findNext("td").text.strip()
        fortuneURLs_df['adwordsBudget'][i] = soup.find("td",text="Adwords budget (in USD)").findNext("td").text.strip()
        fortuneURLs_df['is_run'][i] = 3
        print(f'{Fore.GREEN}\033[1m: 3...{Style.RESET_ALL}', end = '')
        print(f'{Fore.GREEN}\033[1m: Just Completed{Style.RESET_ALL}')
    except:
        print(f'{Fore.RED}\033[1m: New Scraping Exception 3rd Block{Style.RESET_ALL}')
        continue

    # time delay for ease of search
    if runner < 50:
        sleep(3 + runner/10)
    elif runner < 100:
        sleep(3 + 2 * runner/10)
    elif runner < 200:
        sleep(8)
    elif runner < 300:
        sleep(8 + runner/10)
    else:
        sleep(8 + 2 * runner/10)

0 http://www.walmart.com.websiteoutlook.com: Not Run: Exists
1 http://www.amazon.com.websiteoutlook.com: Not Run: Exists
2 http://www.apple.com.websiteoutlook.com: Not Run: Exists
3 http://www.cvshealth.com.websiteoutlook.com: Not Run: Port Errors
4 http://www.unitedhealthgroup.com.websiteoutlook.com: Not Run: Exists
5 http://www.mckesson.com.websiteoutlook.com: Not Run: Exists
6 http://www.att.com.websiteoutlook.com: Not Run: Exists
7 http://www.amerisourcebergen.com.websiteoutlook.com: Not Run: Exists
8 http://www.google.com.websiteoutlook.com: Not Run: Exists
9 http://www.cigna.com.websiteoutlook.com: Not Run: Exists
10 http://www.cardinal.com.websiteoutlook.com: Not Run: Exists
11 http://www.microsoft.com.websiteoutlook.com: Not Run: Exists
12 http://www.marathonpetroleum.com.websiteoutlook.com: Not Run: Exists
13 http://www.homedepot.com.websiteoutlook.com: Not Run: Exists
14 http://www.comcastcorporation.com.websiteoutlook.com: New Scraping Exception 1st Block
15 http://www.anthe

128 http://www.lithia.com.websiteoutlook.com: 1...: New Scraping Exception 2nd Block
129 http://www.amfam.com.websiteoutlook.com: Not Run: Exists
130 http://www.farmers.com.websiteoutlook.com: Not Run: Exists
131 http://www.lkqcorp.com.websiteoutlook.com: Not Run: Port Errors
132 http://www.sempra.com.websiteoutlook.com: Not Run: Exists
133 http://www.centerpointenergy.com.websiteoutlook.com: Not Run: Exists
134 http://www.quantaservices.com.websiteoutlook.com: 1...: New Scraping Exception 2nd Block
135 http://www.group1auto.com.websiteoutlook.com: Not Run: Exists
136 http://www.unum.com.websiteoutlook.com: Not Run: Exists
137 http://www.autozone.com.websiteoutlook.com: 1...: New Scraping Exception 2nd Block
138 http://www.pacificlife.com.websiteoutlook.com: Not Run: Exists
139 http://www.aboutschwab.com.websiteoutlook.com: Not Run: Exists
140 http://www.crowncork.com.websiteoutlook.com: Not Run: Port Errors
141 http://www.ally.com.websiteoutlook.com: Not Run: Exists
142 http://www.lab

232 http://www.abm.com.websiteoutlook.com: 1...: New Scraping Exception 2nd Block
233 http://www.activisionblizzard.com.websiteoutlook.com: Not Run: Port Errors
234 http://www.saic.com.websiteoutlook.com: Not Run: Port Errors
235 http://www.zoetis.com.websiteoutlook.com: Not Run: Exists
236 http://www.ascenaretail.com.websiteoutlook.com: Not Run: Exists
237 http://www.firstam.com.websiteoutlook.com: Not Run: Exists
238 http://www.graphicpkg.com.websiteoutlook.com: Not Run: Exists
239 http://www.olin.com.websiteoutlook.com: Not Run: Port Errors
240 http://www.roberthalf.com.websiteoutlook.com: Not Run: Exists
241 http://www.amtd.com.websiteoutlook.com: Not Run: Port Errors
242 http://www.analog.com.websiteoutlook.com: Not Run: Exists
243 http://www.rushenterprises.com.websiteoutlook.com: Not Run: Exists
244 http://www.fbhs.com.websiteoutlook.com: Not Run: Exists
245 http://www.levistrauss.com.websiteoutlook.com: Not Run: Port Errors
246 http://www.crowncastle.com.websiteoutlook.com: Not

358 http://www.pugetenergy.com.websiteoutlook.com: New Scraping Exception 1st Block
359 http://www.graniteconstruction.com.websiteoutlook.com: Not Run: Exists
360 http://www.skyworksinc.com.websiteoutlook.com: Not Run: Exists
361 http://www.hologic.com.websiteoutlook.com: Not Run: Exists
362 http://www.mscdirect.com.websiteoutlook.com: Not Run: Port Errors
363 http://www.synopsys.com.websiteoutlook.com: Not Run: Exists
364 http://www.boydgaming.com.websiteoutlook.com: Not Run: Exists
365 http://www.stericycle.com.websiteoutlook.com: Not Run: Exists
366 http://www.craneco.com.websiteoutlook.com: Not Run: Exists
367 http://www.cinemark.com.websiteoutlook.com: Not Run: Port Errors
368 http://www.autodesk.com.websiteoutlook.com: Not Run: Exists
369 http://www.zionsbancorporation.com.websiteoutlook.com: Not Run: Port Errors
370 http://www.sci-corp.com.websiteoutlook.com: Not Run: Exists
371 http://www.varian.com.websiteoutlook.com: Not Run: Exists
372 http://www.pricesmart.com.websiteoutloo

In [153]:
print('Total Websites:' + str(len(fortuneURLs_df['is_run'])))
print('Scraping Completed:' + str(sum(1 for r in fortuneURLs_df['is_run'] if r >= 1)))
print('\tAll 3 Blocks Completed:' + str(sum(1 for r in fortuneURLs_df['is_run'] if r == 3)))
print('\t2 Blocks Completed:' + str(sum(1 for r in fortuneURLs_df['is_run'] if r == 2)))
print('\tOnly 1 Block Completed:' + str(sum(1 for r in fortuneURLs_df['is_run'] if r == 1)))
print('Not Run/Incomplete:'+ str(sum(1 for r in fortuneURLs_df['is_run'] if r == 0)))
print('Port Exceptions:'+ str(sum(1 for r in fortuneURLs_df['is_run'] if r < 0)))

Total Websites:423
Scraping Completed:348
	All 3 Blocks Completed:0
	2 Blocks Completed:0
	Only 1 Block Completed:348
Not Run/Incomplete:13
Port Exceptions:62


In [154]:
# df specifications

#fortuneURLs_df.shape               # column row count
fortuneURLs_df.head()              # data peak
#fortuneURLs_df.describe()          # statistic details
#fortuneURLs_df.info()              # column list
#fortuneURLs_df.isna().any()        # check NA

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees,URLs,wout,alexaRank,...,pageSize,codeToTextRatio,semrushRank,keywords,organicTraffic,cost,adwordsKeywords,adwordsTraffic,adwordsBudget,is_run
0,1,Walmart,523964.0,14881.0,236495.0,321803.3,2200000,http://www.walmart.com,http://www.walmart.com.websiteoutlook.com,126,...,64Kb,0.68%,15,29145287,190314352,124437628$,37053,3949947,2396610$,1
1,2,Amazon.com,280522.0,11588.0,225248.0,970680.1,798000,http://www.amazon.com,http://www.amazon.com.websiteoutlook.com,14,...,64Kb,0.27%,3,91384723,950416767,724598977$,79995,2943519,9398034$,1
2,4,Apple,260174.0,55256.0,338516.0,1112640.8,137000,http://www.apple.com,http://www.apple.com.websiteoutlook.com,51,...,64Kb,13.71%,20,17897033,159102793,220655359$,15095,3382490,3923068$,1
3,5,CVS Health,256776.0,6634.0,222449.0,77375.8,290000,http://www.cvshealth.com,http://www.cvshealth.com.websiteoutlook.com,31978,...,64Kb,23.26%,,,,,,,,-1
4,7,UnitedHealth Group,242155.0,13839.0,173889.0,236555.2,325000,http://www.unitedhealthgroup.com,http://www.unitedhealthgroup.com.websiteoutloo...,25580,...,64Kb,18.54%,10993,15919,133787,459981$,173,2898,1324$,1


### <font style = 'italic'> eof

In [155]:
# export processed sample set
fortuneURLs_df.to_csv('op_Final.csv')